# 1. Latent Dirichlet Allocation
LDA는 문서 생성 과정을 역추적    
'문서는 토픽 혼합으로 구성되며, 토픽은 확률분포에 기반해 단어를 생성한다'는 가정   
토픽 제목을 정하진 않으나 결과로부터 토픽 판단 가능   

<null>|내용
:--|:--
문서1|저는 사과랑 바나나를 먹어요   
문서2|우리는 귀여운 강아지가 좋아요   
문서3|저의 깜찍하고 귀여운 강아지가 바나나를 먹어요   

<null>|토픽 분포
:--|:--
문서1|토픽 A 100%   
문서2|토픽 B 100%   
문서3|토픽 B 60%, 토픽 A 40%   

<null>|토픽 단어 분포
:--|:--
토픽A|사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜직하고 0%, 좋아요 0%   
토픽B|사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%   

# 2. LDA 가정
> **1) 사용할 단어 개수를 정함**   

> **2) 사용할 토픽 혼합을 확률분포에 기반해 결정**   
> 예) 토픽이 2개면 강아지 토픽 60%, 과일 토픽 40%

> **3) 문서에 사용할 각 단어를 정함**   
> 1. 토픽 분포에서 토픽을 확률적으로 선택   
> 2. 선택 토픽에서 단어 출현 확률분포에 기반해 문서에 사용할 단어를 고름   
> 예) 강아지 토픽에서는 33% 확률로 단어 '강아지' 선택

# 3. LDA 수행
> **1) 토픽 개수 k 입력**   
> 토픽 k개가 전체 문서에 분포됨을 가정

> **2) 모든 단어를 한 토픽에 랜덤 할당**   
> 각 문서는 토픽을, 토픽은 단어 분포를 갖게 됨

> **3) 모든 단어에 대해 iteration**   
> 해당 단어는 잘못된 토픽, 다른 단어는 올바른 토픽에 할당됨을 가정해 **해당 단어의 토픽 재할당**   

> 예) 문서1 단어 apple 토픽 결정   

> doc1|apple|banana|apple|dog|dog
> :-:|:-:|:-:|:-:|:-:|:-:
> topic|B|B|???|A|A

> doc2|cute|book|king|apple|apple
> :-:|:-:|:-:|:-:|:-:|:-:
> topic|B|B|B|B|B

> 기준1: 해당 문서에서 전체 단어 토픽 확인   
> 단어 apple은 어떤 토픽에도 속할 수 있음   

> doc1|**apple**|**banana**|apple|**dog**|**dog**
> :-:|:-:|:-:|:-:|:-:|:-:
> topic|**B**|**B**|???|**A**|**A**

> 기준2: 전체 문서에서 해당 단어 토픽 확인   
> 단어 apple은 토픽 B일 가능성이 높음

> doc1|**apple**|banana|**apple**|dog|dog
> :-:|:-:|:-:|:-:|:-:|:-:
> topic|**B**|B|**???**|A|A

> doc2|cute|book|king|**apple**|**apple**
> :-:|:-:|:-:|:-:|:-:|:-:
> topic|B|B|B|**B**|**B**

# 4. LSA와 LDA 
LSA는 SVD의 근접 단어들을 토픽으로 묶음   
LDA는 **문서의 특정 토픽 존재 확률**과 **단어가 특정 토픽에 속할 확률**을 추정해 토픽 추출

# 5. 실습
> **1) 정수 인코딩과 단어 집합 생성**   

In [1]:
# 뉴스그룹 데이터 로드
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# 텍스트 전처리
news_df = pd.DataFrame({'document':documents})

news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

<ipython-input-1-aedc00f00276>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [2]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [3]:
# 정수 인코딩 및 단어 빈도수 기록
from gensim import corpora 
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [4]:
print(dictionary[55])

accept


In [5]:
len(dictionary)

64281

> **2) LDA 모델 훈련**

In [6]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.021*"window" + 0.012*"motif" + 0.011*"server" + 0.009*"application"')
(1, '0.011*"people" + 0.009*"would" + 0.007*"government" + 0.005*"state"')
(2, '0.015*"rockefeller" + 0.015*"circuits" + 0.008*"suck" + 0.007*"believer"')
(3, '0.028*"output" + 0.025*"entry" + 0.013*"line" + 0.013*"build"')
(4, '0.016*"privacy" + 0.014*"technology" + 0.013*"encryption" + 0.010*"information"')
(5, '0.020*"would" + 0.017*"like" + 0.015*"know" + 0.010*"think"')
(6, '0.041*"file" + 0.025*"program" + 0.018*"windows" + 0.016*"files"')
(7, '0.018*"game" + 0.016*"year" + 0.016*"team" + 0.012*"games"')
(8, '0.016*"period" + 0.013*"play" + 0.010*"power" + 0.009*"goal"')
(9, '0.012*"guns" + 0.008*"cars" + 0.008*"rate" + 0.008*"firearms"')
(10, '0.015*"armenian" + 0.014*"said" + 0.014*"armenians" + 0.012*"people"')
(11, '0.012*"cross" + 0.010*"kent" + 0.009*"plane" + 0.009*"slave"')
(12, '0.010*"mormons" + 0.009*"mormon" + 0.008*"francis" + 0.007*"weiss"')
(13, '0.018*"drive" + 0.014*"card" + 0.012*"syste

> **4) 문서 토픽 분포 확인**

In [7]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(1, 0.31363308), (2, 0.07571052), (4, 0.018783059), (10, 0.13983168), (14, 0.40638313), (18, 0.034168553)]
1 번째 문서의 topic 비율은 [(5, 0.44155407), (7, 0.06759918), (14, 0.44030643), (16, 0.029455788)]
2 번째 문서의 topic 비율은 [(1, 0.48116532), (5, 0.29229742), (10, 0.06274679), (13, 0.102307916), (14, 0.04893566)]
3 번째 문서의 topic 비율은 [(0, 0.18933143), (1, 0.08074704), (4, 0.095139876), (5, 0.28190193), (13, 0.05666134), (16, 0.28526068)]
4 번째 문서의 topic 비율은 [(6, 0.09158193), (7, 0.87507576)]


In [8]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), 
                                                            topic_list]), ignore_index=True)
            else:
                break
                
    return(topic_table)

In [9]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,14.0,0.4064,"[(1, 0.31362814), (2, 0.07571132), (4, 0.01878..."
1,1,5.0,0.4416,"[(5, 0.44157615), (7, 0.06758654), (14, 0.4402..."
2,2,1.0,0.4812,"[(1, 0.48115027), (5, 0.29228505), (10, 0.0627..."
3,3,16.0,0.2852,"[(0, 0.18951279), (1, 0.08000005), (4, 0.09540..."
4,4,7.0,0.8751,"[(6, 0.09158051), (7, 0.87507725)]"
5,5,14.0,0.6073,"[(5, 0.1592673), (14, 0.6072812), (15, 0.196451)]"
6,6,13.0,0.5040,"[(0, 0.029830465), (1, 0.024769574), (3, 0.077..."
7,7,5.0,0.3552,"[(1, 0.20315629), (5, 0.3552327), (10, 0.20329..."
8,8,1.0,0.4103,"[(1, 0.41026), (5, 0.28741318), (8, 0.06453287..."
9,9,5.0,0.9136,"[(5, 0.91355413), (8, 0.015192115), (16, 0.059..."
